In [1]:
import pyodbc 
import os

sql_conn_str = os.getenv('MSSQL_CONN_STRING_DOCKER') or ''
cnxn = pyodbc.connect(sql_conn_str)

In [ ]:
def find_children(node, rows):
    appending=True
    children = []
    ids = set([node[0]])
    while appending:
        length = len(children)
        children_nodes = [r for r in rows if r[1] in ids]        
        children.extend(children_nodes)
        children = list(set([ tuple(p) for p in children]))
        appending = (length != len(children))
    return children


def build_main_equipment_reverse_lookup():
    sql = """
      SELECT 
          mq.ID, mq.Parent_ID, mq.isMainEquipment, mo.ID as Model_ID, mq.TechnicalInfoTag_ID
      FROM 
          ModelHierarchy mq left join model mo on mo.ModelHierarchy_ID = mq.ID
       """
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    rows = cursor.fetchall() 

    print(f"Total number of models in IOC: {len(rows)}")
    rows = [list(x) for x in rows]
    rows_main_equipment = [r for r in rows if r[2]]
    print(f"Main equipment models in IOC: {len(rows_main_equipment)}")

    reverse_results = {}
    for parent in rows_main_equipment:
        # Reverse lookup dictionary
        for x in find_children(parent, rows):
            reverse_results[x[3]]=parent[0] 
    print(f"Models with a parent main equipment: {len(reverse_results)}")
    return reverse_results


def find_stid_tag(model_hierarchy_id):
    sql = f"""
        SELECT t.*
        FROM TechnicalInfoTag t, ModelHierarchy m
        WHERE m.id = '{model_hierarchy_id}' 
            and t.id = m.TechnicalInfoTag_ID
        """
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    rows = cursor.fetchall() 
    if not rows:
        raise Exception(f"Cannot find stid tag for Model hierarchy:{model_hierarchy_id}")
    else:
        return rows[0]

    
def find_stid_children(stid_id):
    # This takes 16'' too slow, 3M table
    sql=f"""
        WITH tit_children AS (
            SELECT       
                ID, 
                Tag,
                Parent_ID
            FROM       
                TechnicalInfoTag
            WHERE 
                ID = '{stid_id}'
            UNION ALL
            SELECT 
                ti.ID, 
                ti.Tag,
                ti.Parent_ID
            FROM 
                TechnicalInfoTag ti
                INNER JOIN tit_children tic 
                    ON tic.ID = ti.Parent_ID
        )
        SELECT *
        FROM 
            tit_children tic
    """
    
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    rows = cursor.fetchall()
    return rows


def find_notifications(stid_tags):
    str_stid_tags = "','".join(stid_tags)
    sql = f"""
        SELECT * 
        FROM  Notification n
        WHERE n.TechnicalInfoTag_ID in ('{str_stid_tags}')
    """
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    
    columns = [column[0] for column in cursor.description]
    results = []
    for row in cursor.fetchall():
        results.append(dict(zip(columns, row)))
    return results
    
    
    
reverse_me_lookup = build_main_equipment_reverse_lookup()

In [ ]:
from pprint import pprint

cnxn = pyodbc.connect(sql_conn_str)
model_id  = ""
try:
    main_equipment_parent_hierarchy_id = reverse_me_lookup[model_id]
except KeyError:
    print(f"Can't find Main equipment for model {model_id}")

stid_tag = find_stid_tag(main_equipment_parent_hierarchy_id)

stid_children = find_stid_children(stid_tag[0])




In [ ]:
notifications = find_notifications([x[0] for x in stid_children])
print(f"Found:{len(notifications)}")
pprint(notifications)
